In [ ]:
from __future__ import annotations

import itertools
from pathlib import Path

In [ ]:
type Position = tuple[int, int]
type Grid = dict[Position, str]


class Guard:
    """A guard that moves around a grid, turning right when blocked."""

    def __init__(self, pos: Position, extra_obstruction: Position | None = None):
        """Create a guard at the given position, with a given heading."""
        self.pos = pos
        self.headings = itertools.cycle(["^", ">", "v", "<"])
        self.heading = next(self.headings)
        self.visited: set[tuple[Position, str]] = set()
        self.extra_obstruction = extra_obstruction

    def on_map(self, grid: Grid) -> bool:
        """Return True if the guard is on the grid."""
        return self.pos in grid

    def next_pos(self) -> Position:
        """Return the position the guard will move to next."""
        row, col = self.pos
        if self.heading == "^":
            return row - 1, col
        if self.heading == ">":
            return row, col + 1
        if self.heading == "v":
            return row + 1, col
        if self.heading == "<":
            return row, col - 1
        raise ValueError(f"Invalid heading: {self.heading}")

    def is_blocked(self, grid: Grid) -> bool:
        """Return True if the guard is blocked from moving forward."""
        next_pos = self.next_pos()
        return grid.get(next_pos) == "#" or next_pos == self.extra_obstruction

    def move_or_turn(self, grid: Grid):
        """If blocked, turn right. Otherwise, move one step forward."""
        if (self.pos, self.heading) in self.visited:
            # For part 2
            raise RecursionError("Guard is stuck in a loop")
        self.visited.add((self.pos, self.heading))
        if self.is_blocked(grid):
            self.heading = next(self.headings)
        else:
            self.pos = self.next_pos()

In [ ]:
# Read the input
grid: Grid = {}

with Path("day06_input.txt").open() as file:
    for row, line in enumerate(file):
        for col, char in enumerate(line.strip()):
            pos = row, col
            grid[pos] = char
            if char not in ".#":
                guard_start_pos = row, col

# Part 1


In [ ]:
# Move the guard until it walks off the grid.
guard = Guard(guard_start_pos)
while guard.on_map(grid):
    guard.move_or_turn(grid)

# Number of unique positions visited by the guard.
visited_positions = {pos for pos, _heading in guard.visited}

len(visited_positions)

# Part 2


In [ ]:
obstruction_positions = []
for extra_obstruction in visited_positions:
    # Try placing an obstruction at each visited position from the original run, and
    # then re-run the guard from its starting position. See if that causes the guard to
    # get stuck in a loop.
    try:
        guard = Guard(guard_start_pos, extra_obstruction)
        while guard.on_map(grid):
            guard.move_or_turn(grid)
    except RecursionError:
        obstruction_positions.append(extra_obstruction)

len(obstruction_positions)